In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Installs

In [2]:
!pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0
!pip install datasets==2.16.1 PyPDF2==3.0.1 langchain_experimental python-dotenv==1.0.1 pyyaml==6.0.1 coloredlogs==15.0.1 mdc==1.2.1 pytest==8.1.2
!pip install -r drive/MyDrive/RAG-llamaindex/requirements.txt
!pip install huggingface_hub
!pip install llama-index-llms-huggingface
!pip uninstall -y bitsandbytes accelerate transformers
!pip install bitsandbytes accelerate transformers
!pip install gradio
!pip install spaces

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

#Imports


In [4]:
# For Loading the LLM
import json
import torch
import time
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TextIteratorStreamer,
                          pipeline)

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

import warnings
warnings.filterwarnings('ignore')

# For Data Ingestion
from typing import Literal, Any, List
import logging
import PyPDF2
import sys
sys.path.append('drive/MyDrive/RAG-llamaindex')
import os
from langchain_experimental.text_splitter import SemanticChunker
from math import ceil

# For Embeddings
from transformers import AutoModel

# For the LLM Model
from llama_index.llms.huggingface import HuggingFaceLLM
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# For deployment
import gradio as gr
import spaces
from threading import Thread

#Loading the LLM

In [ ]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Getting the Hugging Face token
config_data = json.load(open("drive/MyDrive/RAG-llamaindex/config.json"))
HF_TOKEN = config_data["HF_TOKEN"]

# Defining the pre-trained model we will use, which is Llama-3-8B
#model_name = "meta-llama/Meta-Llama-3-8B"
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"


# Defining the quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Defing the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name ,
                                          token=HF_TOKEN)
#tokenizer.pad_token = tokenizer.eos_token

# Initialising the LLM
model = AutoModelForCausalLM.from_pretrained(
    model_name ,
    torch_dtype=torch.bfloat16, #added this
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

#Data Ingestion

In [ ]:
from typing import Literal, Any, List
import logging
import PyPDF2
import sys
sys.path.append('drive/MyDrive/RAG-llamaindex')
import os
from langchain_experimental.text_splitter import SemanticChunker
from math import ceil

In [10]:
reader = SimpleDirectoryReader(input_dir="drive/MyDrive/RAG-llamaindex/Data")
documents = reader.load_data(num_workers=4)

# Filter out the data where the file_name is not "Harry Potter The Complete Collection.pdf"
documents = [doc for doc in documents if doc.metadata["file_name"] == "Harry Potter The Complete Collection.pdf"]

# Creating a function for chunking using the LLama Index SentenceSplitter
def get_chunks(
    documents: list[Any],
    chunk_size: int = 512,
    chunk_overlap: float = 0.0
) -> list[str]:
    overlap = int(chunk_size * chunk_overlap)
    text_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    chunks= text_splitter.get_nodes_from_documents(documents, show_progress=True)

    return chunks

# Creating a function for manual chunking
def get_chunks_manual(documents: list[Any], chunk_size: int = 512, chunk_overlap: float = 0.0) -> List[str]:
    """
    Takes in a `file_path`, retrieves the document, breaks it down into chunks of size
    `chunk_size` and overlap `chunk_overlap`, and returns the chunks.
    """
    chunks = []

    text = ""
    for doc in documents:
        text += doc.text
    # replace every \t with " "
    text = text.replace("\t", " ")

    step_size = ceil(chunk_size * (1 - chunk_overlap))
    for i in range(0, len(text), step_size):
        chunks.append(text[i:i + chunk_size])

    return chunks

KeyboardInterrupt: 

In [ ]:
# Using the Sentence Splitter

# Creating chunks of 512 characters with no overlap
chunks_512_0 = get_chunks(documents)
print("Created", len(chunks_512_0), "chunks of size 512 and 0% overlap with the SentenceSplitter")
print("The 100th chunk is:", chunks_512_0[100])

# Creating chunks of 512 with 50% overlap
chunks_512_50 = get_chunks(documents, chunk_size=512, chunk_overlap=0.5)
print("Created", len(chunks_512_50), "chunks of size 512 and 0% overlap with the SentenceSplitter")
print("The 100th chunk is:", chunks_512_50[100])

In [ ]:
# Using the Manual Chunking

# Creating chunks of 512 with no overlap
manual_chunks_512_0 = get_chunks_manual(documents, 512, 0.0)
print("Created", len(manual_chunks_512_0), "chunks of size 512 and 0% overlap with the Manual Splitter")
print("The 100th chunk is:", manual_chunks_512_0[100])

# Creating chunks of 512 with 50% overlap
manual_chunks_512_50 = get_chunks_manual(documents, 512, 0.5)
print("Created", len(manual_chunks_512_50), "chunks of size 512 and 50% overlap with the Manual Splitter")
print("The 100th chunk is:", manual_chunks_512_50[100])

Created 12269 chunks of size 512 and 0% overlap with the Manual Splitter
The 100th chunk is: en the bill, snorted in disgust, and flipped over the
postcard.
“Marge’s ill,” he informed Aunt Petunia. “Ate a funny whelk . . .”
“Dad!” said Dudley suddenly. “Dad, Harry’s got something!”
Harry was on the point of unfolding his letter, which was written on the
same heavy parchment as the envelope, when it was jerked sharply out of his
hand by Uncle Vernon.
“That’s 
mine
!” said Harry, trying to snatch it back.
“Who’d be writing to you?” sneered Uncle Vernon, shaking the letter open
with one hand and glanc
Created 24537 chunks of size 512 and 0% overlap with the Manual Splitter
The 100th chunk is: and took out the silver Put-Outer. He clicked it once, and twelve balls
of light sped back to their street lamps so that Privet Drive glowed suddenly
orange and he could make out a tabby cat slinking around the corner at the
other end of the street. He could just see the bundle of blankets on the ste

#Creating the Embedding Model

In [ ]:
from transformers import AutoModel

In [ ]:
class HuggingFaceEmbedding:
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def get_text_embedding(self, text: str) -> torch.Tensor:
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze()

    def get_embeddings(self, texts: list[str], batch_size: int = 16) -> list[torch.Tensor]:
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            inputs = self.tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)
            embeddings.extend(batch_embeddings.cpu())
        return embeddings

embed_model =  HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


#Defining the LLM Model

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
llm= HuggingFaceLLM(model=model, tokenizer=tokenizer)

# Find the embedding of each text in Manual_Chunks of size 512 and 0% overlap and put in list
manual_512_0_embed = embed_model.get_embeddings(manual_chunks_512_0)
manual_512_0_embed_np = [embedding.numpy() for embedding in manual_512_0_embed]
print(len(manual_512_0_embed_np))

# Find the embedding of each text in Manual_Chunks of size 512 and 50% overlap and put in list
manual_512_50_embed = embed_model.get_embeddings(manual_chunks_512_50)
manual_512_50_embed_np = [embedding.numpy() for embedding in manual_512_50_embed]
print(len(manual_512_50_embed_np))

12269


KeyboardInterrupt: 

In [ ]:
# Creating the first Retriever: Top-k Embeddings Retriever

def top_k_embed_retriever(query: str, k: int, embed_model, manual_embed_np, texts):
    query_embed = embed_model.get_text_embedding(query)
    query_embed_np = query_embed.cpu().numpy().reshape(1, -1)

    # Calculate cosine similarity with all the other embeddings
    similarities = cosine_similarity(query_embed_np, manual_embed_np).flatten()

    # Find the top k indices
    top_k_indices = np.argpartition(similarities, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]

    # Retrieve the top k texts
    top_k_texts = [texts[i] for i in top_k_indices]

    return top_k_texts
"""
# Example usage
query = "Hogwarts"
k = 3
top_k_texts = top_k_embed_retriever(query, k, embed_model, np.stack(manual_embed_np), manual_chunks)
print(top_k_texts[0])
print("___________")
print(top_k_texts[1])
print("___________")
print(top_k_texts[2])"""

In [ ]:
vectorizer_512_0 = TfidfVectorizer()
vectorizer_512_50 = TfidfVectorizer()

tfidf_matrix_512_0 = vectorizer_512_0.fit_transform(manual_chunks)
tfidf_matrix_512_50 = vectorizer_512_50.fit_transform(manual_chunks_512_50)

NameError: name 'manual_chunks' is not defined

In [ ]:
# Creating the second Retriever: Top-k TF-IDF Retriever
def top_k_tfidf_retriever(query: str, k: int, tfidf_matrix, texts, vectorizer):
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_k_indices = np.argpartition(similarities, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]
    top_k_texts = [texts[i] for i in top_k_indices]
    return top_k_texts

"""# Example usage
query = "Hogwarts"
k = 3
top_k_texts = top_k_tfidf_retriever(query, k, tfidf_matrix_512_0, manual_chunks, vectorizer_512_0)
print(top_k_texts[0])
print("___________")
print(top_k_texts[1])
print("___________")
print(top_k_texts[2])"""

#Creating the Pipeline

In [ ]:
import pandas as pd

In [15]:
SYS_PROMPT = """You are an assistant for answering questions about the Harry Potter books.
You are given the extracted parts of the books and a question. Provide a short answer.
If you don't know the answer, just say "I do not know." Don't make up an answer."""

def format_prompt(prompt,retrieved_documents,k):
  """using the retrieved documents we will prompt the model to generate our responses"""
  PROMPT = f"Question:{prompt}\nContext:"
  for idx in range(k) :
    PROMPT+= f"{retrieved_documents[idx]}\n"
  return PROMPT

def generate(formatted_prompt):
  formatted_prompt = formatted_prompt[:2000] # to avoid GPU OOM
  messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":formatted_prompt}]
  # tell the model to generate
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)
  outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:

def rag_pipeline(documents: list[Any], df: pd.DataFrame, chunk_sz: int = 512, chunk_ol: float = 0.0, retriever: str = "embed", k: int = 3, embd_mod=None):
    '''
    This function takes a dataframe `df` that contains the QA's, `chunk_sz` and overlap `chunk_ol`,
     a retriver `retiever` between the top_k_embed_retriever and the top_k_tfidf_retriever,
    the parameter `k` for the retrievers and an embeddings model.
    '''
    # Creating the chunks
    chunks = get_chunks_manual(documents, chunk_sz, chunk_ol)

    # Manual Embeddings for TF-IDF
    if retriever == 'embed':
      # Loading the embedding model and embed the chunks
      manual_embed = embed_model.get_embeddings(chunks)
      manual_embed_np = [embedding.numpy() for embedding in manual_embed]
    else:
      # Calculating TF-IDF
      vectorizer = TfidfVectorizer()
      tfidf_matrix = vectorizer.fit_transform(chunks)

    results = []
    for index, row in df.iterrows():
      query = row["Question"]

      if retriever == "embed":
        top_k_texts = top_k_embed_retriever(query, k, embed_model, np.stack(manual_embed_np), chunks)
      else:
        top_k_texts = top_k_tfidf_retriever(query, k, tfidf_matrix, chunks, vectorizer)

      retrieved_context = " ".join(top_k_texts)

      formatted_prompt = format_prompt(query,top_k_texts,k)
      llm_answer = generate(formatted_prompt)

      result = {
          "Question": query,
          "True_Answer": row['Answer'],
           "Retrieved_Context": retrieved_context,
           "LLM_Answer": llm_answer
      }
      results.append(result)

    return pd.DataFrame(results)


In [21]:
# Example
prompt = "Who is Harry's best friend"
k = 3
top_k_texts = top_k_embed_retriever(prompt, k, embed_model, np.stack(manual_512_0_embed_np), manual_chunks_512_0)
formatted_prompt = format_prompt(prompt,top_k_texts,k)
llm_answer = generate(formatted_prompt)

print(llm_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


According to the context, Harry's best friends are Ron Weasley and Hermione Granger.


#Creating a Harry Potter Chatbot

In [ ]:
# Getting the Hugging Face token
config_data = json.load(open("drive/MyDrive/RAG-llamaindex/config.json"))
HF_TOKEN = config_data["HF_TOKEN"]

# Defining the pre-trained model we will use, which is Llama-3-8B
#model_name = "meta-llama/Meta-Llama-3-8B"
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# Defining the quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Defing the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name ,
                                          token=HF_TOKEN)
#tokenizer.pad_token = tokenizer.eos_token

# Initialising the LLM
model = AutoModelForCausalLM.from_pretrained(
    model_name ,
    torch_dtype=torch.bfloat16, #added this
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
reader = SimpleDirectoryReader(input_dir="drive/MyDrive/RAG-llamaindex/Data")
documents = reader.load_data(num_workers=4)

# Creating a function for manual chunking
def get_chunks_manual(documents: list[Any], chunk_size: int = 512, chunk_overlap: float = 0.0) -> List[str]:
    """
    Takes in a `file_path`, retrieves the document, breaks it down into chunks of size
    `chunk_size` and overlap `chunk_overlap`, and returns the chunks.
    """
    chunks = []

    text = ""
    for doc in documents:
        text += doc.text
    # replace every \t with " "
    text = text.replace("\t", " ")

    step_size = ceil(chunk_size * (1 - chunk_overlap))
    for i in range(0, len(text), step_size):
        chunks.append(text[i:i + chunk_size])

    return chunks

# Creating chunks of 512 with no overlap
manual_chunks_512_0 = get_chunks_manual(documents, 512, 0.0)
print("Created", len(manual_chunks_512_0), "chunks of size 512 and 0% overlap with the Manual Splitter")
print("The 100th chunk is:", manual_chunks_512_0[100])

#Embed
class HuggingFaceEmbedding:
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def get_text_embedding(self, text: str) -> torch.Tensor:
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze()

    def get_embeddings(self, texts: list[str], batch_size: int = 16) -> list[torch.Tensor]:
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            inputs = self.tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)
            embeddings.extend(batch_embeddings.cpu())
        return embeddings

embed_model =  HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Find the embedding of each text in Manual_Chunks of size 512 and 0% overlap and put in list
manual_512_0_embed = embed_model.get_embeddings(manual_chunks_512_0)
manual_512_0_embed_np = [embedding.numpy() for embedding in manual_512_0_embed]
print(len(manual_512_0_embed_np))

# Creating the first Retriever: Top-k Embeddings Retriever
def top_k_embed_retriever(query: str, k: int, embed_model, manual_embed_np, texts):
    query_embed = embed_model.get_text_embedding(query)
    query_embed_np = query_embed.cpu().numpy().reshape(1, -1)

    # Calculate cosine similarity with all the other embeddings
    similarities = cosine_similarity(query_embed_np, manual_embed_np).flatten()

    # Find the top k indices
    top_k_indices = np.argpartition(similarities, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]

    # Retrieve the top k texts
    top_k_texts = [texts[i] for i in top_k_indices]

    return top_k_texts


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Created 12419 chunks of size 512 and 0% overlap with the Manual Splitter
The 100th chunk is: en the bill, snorted in disgust, and flipped over the
postcard.
“Marge’s ill,” he informed Aunt Petunia. “Ate a funny whelk . . .”
“Dad!” said Dudley suddenly. “Dad, Harry’s got something!”
Harry was on the point of unfolding his letter, which was written on the
same heavy parchment as the envelope, when it was jerked sharply out of his
hand by Uncle Vernon.
“That’s 
mine
!” said Harry, trying to snatch it back.
“Who’d be writing to you?” sneered Uncle Vernon, shaking the letter open
with one hand and glanc


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

In [14]:
import spaces
from threading import Thread

SYS_PROMPT = """You are an assistant for answering questions about the Harry Potter books.
You are given the extracted parts of the books and a question. Provide a short answer.
If you don't know the answer, just say "I do not know." Don't make up an answer."""

def format_prompt(prompt,retrieved_documents,k):
  """using the retrieved documents we will prompt the model to generate our responses"""
  PROMPT = f"Question:{prompt}\nContext:"
  for idx in range(k) :
    PROMPT+= f"{retrieved_documents[idx]}\n"
  return PROMPT


@spaces.GPU(duration=150)
def talk(prompt,history):
    k = 1 # number of retrieved documents

    top_k_texts = top_k_embed_retriever(prompt, k, embed_model, np.stack(manual_512_0_embed_np), manual_chunks_512_0)
    formatted_prompt = format_prompt(prompt,top_k_texts,k)
    formatted_prompt = formatted_prompt[:2000] # to avoid GPU OOM

    messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":prompt}]
    # tell the model to generate

    input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
    ).to(model.device)
    outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
    )
    streamer = TextIteratorStreamer(
            tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
        )
    generate_kwargs = dict(
        input_ids= input_ids,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.95,
        temperature=0.75,
        eos_token_id=terminators,
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    outputs = []
    for text in streamer:
        outputs.append(text)
        print(outputs)
        yield "".join(outputs)

TITLE = "Harry Potter Chatbot"

DESCRIPTION = """
This is a chatbot to answer any
"""

demo = gr.ChatInterface(
    fn=talk,
    chatbot=gr.Chatbot(
        show_label=True,
        show_share_button=True,
        show_copy_button=True,
        likeable=True,
        layout="bubble",
        bubble_full_width=False,
    ),
    theme="Soft",
    examples=[["Who is the main character?"]],
    title=TITLE,
    description=DESCRIPTION,

)
demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9e2f0e3c4dc26245de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


['']
['', 'The ']
['', 'The ', 'main ']
['', 'The ', 'main ', 'character ']
['', 'The ', 'main ', 'character ', 'of ']
['', 'The ', 'main ', 'character ', 'of ', 'the ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ', 'Harry ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ', 'Harry ', 'James ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ', 'Harry ', 'James ', '']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ', 'Harry ', 'James ', '', '']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ',

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


['']
['', 'The ']
['', 'The ', 'main ']
['', 'The ', 'main ', 'character ']
['', 'The ', 'main ', 'character ', 'of ']
['', 'The ', 'main ', 'character ', 'of ', 'the ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ', 'Harry ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ', 'Harry ', 'James ']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ', 'Harry ', 'James ', '']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ', 'is ', 'Harry ', 'James ', '', '']
['', 'The ', 'main ', 'character ', 'of ', 'the ', 'Harry ', 'Potter ', 'series ',

...

['']
['', 'Harry ']
['', 'Harry ', '']
['', 'Harry ', '', "Potter's "]
['', 'Harry ', '', "Potter's ", 'best ']
['', 'Harry ', '', "Potter's ", 'best ', 'friend ']
['', 'Harry ', '', "Potter's ", 'best ', 'friend ', 'is ']
['', 'Harry ', '', "Potter's ", 'best ', 'friend ', 'is ', 'Ron ']
['', 'Harry ', '', "Potter's ", 'best ', 'friend ', 'is ', 'Ron ', '']
['', 'Harry ', '', "Potter's ", 'best ', 'friend ', 'is ', 'Ron ', '', '']
['', 'Harry ', '', "Potter's ", 'best ', 'friend ', 'is ', 'Ron ', '', '', '']
['', 'Harry ', '', "Potter's ", 'best ', 'friend ', 'is ', 'Ron ', '', '', '', 'Weasley.']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


['']
['', 'According ']
['', 'According ', 'to ']
['', 'According ', 'to ', 'the ']
['', 'According ', 'to ', 'the ', 'Harry ']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ', 'book ']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ', 'book ', '']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ', 'book ', '', 'series, ']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ', 'book ', '', 'series, ', '']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ', 'book ', '', 'series, ', '', '']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ', 'book ', '', 'series, ', '', '', '']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ', 'book ', '', 'series, ', '', '', '', '']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ', 'book ', '', 'series, ', '', '', '', '', '']
['', 'According ', 'to ', 'the ', 'Harry ', 'Potter ', 'book ', '', 'series, ', '', '', '', '', '', '"Alohomora" ']
['', 'According ', 'to ', 'th

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


['']
['', 'Harry ']
['', 'Harry ', 'Potter ']
['', 'Harry ', 'Potter ', 'married ']
['', 'Harry ', 'Potter ', 'married ', 'Ginny ']
['', 'Harry ', 'Potter ', 'married ', 'Ginny ', '']
['', 'Harry ', 'Potter ', 'married ', 'Ginny ', '', '']
['', 'Harry ', 'Potter ', 'married ', 'Ginny ', '', '', '']
['', 'Harry ', 'Potter ', 'married ', 'Ginny ', '', '', '', 'Weasley.']
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9e2f0e3c4dc26245de.gradio.live
